<a href="https://colab.research.google.com/github/ethansong206/Climate-Plus-Project/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import necessary packages for the code

---

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

---
Load the Duke Dining csv file and make the data easier to handle. This includes but is not limited to:

1) Renaming locations

2) Changing the date from a string to a date object

---

In [2]:
DiningDataFull = pd.read_csv('Climate+ Data 2019 thru 2023.csv', 
                             dtype = {'Priority 1': str, 'Priority 2': str,
                                      'Priority 3': str})
#print(DiningDataFull['Unit Name'].value_counts()) # Show how many entries there are of each location

#below code to make 'Unit Name' column easier to handle
#can add more lines given more locations
def location_rename(location):
    if("Marketplace" in location): #Combine data for Marketplace Kitchen and Marketplace Special Event
        return "Marketplace"
    if("Marine Lab" in location):
        return "DuML"
    if("Trinity" in location):
        return "Trinity"
    if("Freeman" in location):
        return "Freeman"
    return None

DiningDataFull['Unit Name'] = DiningDataFull.apply(lambda d: location_rename(d['Unit Name']), axis = 1)

DiningDataFull['Purchase Date'] = DiningDataFull.apply(lambda d: datetime.strptime(d['Purchase Date'], '%m/%d/%Y').date(), axis = 1)

print("Rows in DiningDataFull: ", DiningDataFull.shape[0])

Rows in DiningDataFull:  90534


---
Extract the unit names for each item into a new column called `Unit`, then rename each unit into a more simple label (i.e. LB CS to LB). Convert cans and bottle cases into the equivalent value in OZ.

Extract the simplified item name from `Vendor Item Description` into a new column called `Item Name`.

---
Note on Exclusion: Some units are left out in this first calculation of emissions as they are either not directly food (i.e. gloves) or are too difficult to go through individually and find a measurement that is not ambiguous. The total number of entries left out is 12171. **The number of non-food items in this amount can be calculated later.**

---
Note on Conversions: The column `Vendor Item Purchase Unit` is in the format x/y n which should be read as x bags of y units of n food.

---

In [3]:
#this code extracts just the unit information
DiningDataFull['Unit'] = DiningDataFull['Vendor Item Purchase Unit'].str.extract(r" ?([A-Za-zÀ-ÿ ]*)$").astype(str)
#below code to simplify redundant labels (i.e. LB CS to LB)
def unit_rename(unit):
    if('LB' in unit) | ('lb' in unit) | ('Lb' in unit) | ('Pound' in unit):
        return 'LB'
    if('OZ' in unit) | ('oz' in unit) | ('Oz' in unit) | (' Z' in unit):
        return 'OZ'
    if('GA' in unit) | ('Gal' in unit):
        return 'GA'
    if('QT' in unit):
        return 'QT'
    if('PT' in unit) | ('Pint' in unit) | ('PINT' in unit):
        return 'PT'
    if('LT' in unit):
        return 'LT' 
    if('BU' in unit) | ('Bushel' in unit): #bushels
        return 'BU'
    if('KG' in unit):
        return 'KG' 
    if('GR' in unit): #grams
        return 'GR' 
    if('ML' in unit) | ('ml' in unit) | ('Ml' in unit): #milliliters
        return 'ML'
    if('CN' in unit) | ('Can' in unit):
        return 'CN'
    if('Bottle Case' in unit):
        return 'Bottle Case'
    return None
#Notes: Bottle Case is 64 oz each, find # of can and translate to oz, anything with EA is not included for now (~9000)
DiningDataFull['Unit'] = DiningDataFull.apply(lambda d: unit_rename(d['Unit']), axis = 1)

#convert cans to OZ
#using estimates for weight of cans through https://food.unl.edu/article/how-interpret-can-size-numbers
DiningDataFull = DiningDataFull.replace({'#10 CN' : '110.5 OZ'}, regex = True)
DiningDataFull = DiningDataFull.replace({'#10 Can' : '110.5 OZ'}, regex = True)
DiningDataFull = DiningDataFull.replace({'#300 CN' : '15 OZ'}, regex = True)
DiningDataFull = DiningDataFull.replace({'CN' : 'OZ'})
#convert bottle case to OZ
DiningDataFull = DiningDataFull.replace({' Bottle Case' : '/64 OZ'}, regex = True)
DiningDataFull = DiningDataFull.replace({'Bottle Case' : 'OZ'})

#WIP
#extract item names

---
Make a new column called `Total Amount` for the total amount of food in the current unit of measurement, before converting to grams. Make a new column called `Total Amount(g)` for converting all units to the equivalent value in grams, then filter out main descriptor word(s) in `Vendor Item Description` into a new column called `Food Name`.

---

Take the column with units in grams and multiply by the `Receive Quantity` if provided into a new column called `Total Grams`. If there is no value in `Receive Quantity`, then assume it is the value 1.

---
Note on Conversions: Most of these conversions are estimated to the nearest tenth. **More accurate calculations can be found later**

---

In [4]:
#make a new dataset for just entries with known units
DDReduced = DiningDataFull[DiningDataFull['Unit'].notna()]
#print(DDReduced.head())

#find total amount of food before converting to grams
DDReduced = DDReduced.copy()
DDReduced['Total Amount'] = DDReduced['Vendor Item Purchase Unit'].str.extract(r'^[a-zA-Z]* ?-?/? ?([0-9]*/?[0-9.]*-?[0-9.]*)')
DDReduced['Range'] = DDReduced['Total Amount'].str.extract(r'([0-9.]*-[0-9.]*)')
DDReduced['Range'] = DDReduced['Range'].astype(str)
DDReduced['Range'] = DDReduced['Range'].replace({'nan' : '0'})
DDReduced['Range Average'] = DDReduced['Range'].replace({'-' : '+'}, regex = True)
DDReduced['Range Average'] = DDReduced.apply(lambda d: eval(d['Range Average']), axis = 1)
DDReduced['Range Average'] /= 2

has_slash = ~(DDReduced['Total Amount'].str.contains('/')) & (DDReduced['Total Amount'].str.len() > 0)
DDReduced.loc[has_slash, 'Total Amount'] = (
    '1/' + DDReduced.loc[has_slash, 'Total Amount']
)

has_range = DDReduced['Total Amount'].str.contains('-')
DDReduced.loc[has_range, 'Total Amount'] = (
    DDReduced.loc[has_range, 'Total Amount'].str.split('/').str[0]
    + '/'
    + DDReduced.loc[has_range, 'Range Average'].astype(str)
)

DDReduced = DDReduced.drop(['Range', 'Range Average'], axis = 1)

DDReduced['Total Amount'] = DDReduced['Total Amount'].replace({'' : '0'})
DDReduced['Receive Quantity'] = DDReduced['Receive Quantity'].fillna(0)
DDReduced = DDReduced[DDReduced['Receive Quantity'] != 0] #get rid of any rows that were not received
DDReduced['Total Amount'] = DDReduced['Total Amount'].replace({'/' : '*'}, regex = True)
DDReduced['Total Amount'] = DDReduced.apply(lambda d: eval(d['Total Amount']), axis = 1)
DDReduced['Total Amount'] = DDReduced['Total Amount'].astype(float) * DDReduced['Receive Quantity'].astype(float)

#convert units to grams
def convert_units(row):
    if row['Unit'] == 'LB':
        return row['Total Amount'] * 453.6
    if row['Unit'] == 'OZ':
        return row['Total Amount'] * 28.35
    if row['Unit'] == 'GA':
        return row['Total Amount'] * 3785.4 #assuming density of water, most drinks are MORE dense so number is underestimate
    if row['Unit'] == 'QT':
        return row['Total Amount'] * 3785.4 / 4
    if row['Unit'] == 'PT':
        return row['Total Amount'] * 3785.4 / 8
    if (row['Unit'] == 'LT') | (row['Unit'] == 'KG'):
        return row['Total Amount'] * 1000 #also assuming density of water, most drinks MORE dense
    if row['Unit'] == 'BU':
        return row['Total Amount'] * 32.5 * 453.6 #average of 40lbs per bu apples, squash, etc. and 25lbs per bu peppers, etc.
    if (row['Unit'] == 'ML') | (row['Unit'] == 'G'):
        return row['Total Amount'] * 453.6
    return 0
DDReduced['Total Amount(g)'] = DDReduced.apply(convert_units, axis = 1)
DDReduced['Total Amount(kg)'] = DDReduced['Total Amount(g)'] / 1000

print("Excluded rows: ", DiningDataFull.shape[0] - DDReduced.shape[0])
#print(DDReduced.head())

<ipython-input-4-78a7d31c9a2b>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DDReduced['Total Amount'] = DDReduced['Total Amount'].replace({'/' : '*'}, regex = True)


Excluded rows:  17099


---
####Load the csv with CO2 numbers and find numbers for carbon emissions

---

In [5]:
CO2 = pd.read_csv('CO2 values for FACCWTHA v1.1 - Foods.csv')

#replace values below if need to update values
#CURRENT VALUES: from CO2 spreadsheet above (Poore and Nemecek 2018) (6/5/23)
beef_emission_ratio = CO2[CO2['FoodDisplayName'] == 'Steak']['CO2eRatioSLBBook'].mean() #Europe average, US is LARGER at ~54kg CO2e/kg
pork_emission_ratio = CO2[(CO2['FoodDisplayName'] == 'Ham') | (CO2['FoodDisplayName'] == 'Sausage')]['CO2eRatioSLBBook'].mean()
poultry_emission_ratio = CO2[CO2['FoodDisplayName'] == 'Chicken']['CO2eRatioSLBBook'].mean() #chicken and turkey, averaged from Europe and US, which is LARGER than just US
lamb_emission_ratio = CO2[CO2['FoodDisplayName'] == 'Lamb']['CO2eRatioSLBBook'].mean() #Europe, slightly higher than global average of ~40
seafood_emission_ratio = CO2[CO2['FoodAisle'] == 'Fish']['CO2eRatioSLBBook'].mean() #average of three common fish
eggs_emission_ratio = CO2[CO2['FoodDisplayName'] == 'Eggs']['CO2eRatioSLBBook'].mean() #world average of eggs
meatsub_emission_ratio = 2.19 #(https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9936781/ , Mejia et al., 2020)
#add more values here as the data gets more clean
#later, replace means/averages with weighted averages depending on how often eaten and average amount eaten each instance

print("beef: ", beef_emission_ratio)
print("pork: ", pork_emission_ratio)
print("poultry: ", poultry_emission_ratio)
print("lamb: ", lamb_emission_ratio)
print("seafood: ", seafood_emission_ratio)
print("eggs: ", eggs_emission_ratio)
print("meatsub: ", meatsub_emission_ratio)

#print(CO2.head())

beef:  46.2
pork:  9.774999999999999
poultry:  6.8
lamb:  43.0
seafood:  5.133333333333334
eggs:  4.7
meatsub:  2.19


---
#CALCULATING CARBON EMISSIONS

Currently includes: Meats, Meat Alternatives, and Eggs

NOTE: 651400 meats, 651401 local meats, processed meat, and specialty meats are excluded as of now, for a total of 3964 rows

---

In [26]:
DDReduced['Product Group'] = DDReduced['Product Group'].str.lower() #make lowercase to simplify code

#print(DDReduced['Product Group'].value_counts())

def calc_emissions(row):
    if row['Product Group'] == 'beef':
        return row['Total Amount(kg)'] * beef_emission_ratio
    if row['Product Group'] == 'pork':
        return row['Total Amount(kg)'] * pork_emission_ratio
    if row['Product Group'] == 'seafood':
        return row['Total Amount(kg)'] * seafood_emission_ratio
    if row['Product Group'] == 'meat substitute':
        return row['Total Amount(kg)'] * meatsub_emission_ratio
    if row['Product Group'] == 'lamb':
        return row['Total Amount(kg)'] * lamb_emission_ratio
    if (row['Product Group'] == 'chicken') | (row['Product Group'] == 'poultry') | (row['Product Group'] == 'turkey') \
    | (row['Product Group'] == 'duck'):
        return row['Total Amount(kg)'] * beef_emission_ratio
    if row['Product Group'] == 'eggs':
        return row['Total Amount(kg)'] * eggs_emission_ratio
    return 0

DDReduced['CO2e(kg)'] = DDReduced.apply(calc_emissions, axis = 1)

total_food_amount_kg = DDReduced['Total Amount(kg)'].sum()
total_food_amount_calced_kg = DDReduced[DDReduced['CO2e(kg)'] != 0]['Total Amount(kg)'].sum()
percent_food_calced = total_food_amount_calced_kg / total_food_amount_kg * 100

print('The percentage of food (by weight) with calculated carbon emissions so far is: ', percent_food_calced, "%")

The percentage of food (by weight) with calculated carbon emissions so far is:  37.66037229036413 %


---
###Making one dataset for each location and each year

---

In [18]:
location_col = DDReduced['Unit Name']
DiningData_Marketplace = DDReduced[location_col == "Marketplace"]
DiningData_DuML = DDReduced[location_col == "DuML"]
DiningData_Trinity = DDReduced[location_col == "Trinity"]
DiningData_Freeman = DDReduced[location_col == "Freeman"]

date_col = DDReduced['Purchase Date']
DiningData_2019 = DDReduced[(date_col >= pd.Timestamp(2019, 1, 1)) & (date_col < pd.Timestamp(2020, 1, 1))]
DiningData_2020 = DDReduced[(date_col >= pd.Timestamp(2020, 1, 1)) & (date_col < pd.Timestamp(2021, 1, 1))]
DiningData_2021 = DDReduced[(date_col >= pd.Timestamp(2021, 1, 1)) & (date_col < pd.Timestamp(2022, 1, 1))]
DiningData_2022 = DDReduced[(date_col >= pd.Timestamp(2022, 1, 1)) & (date_col < pd.Timestamp(2023, 1, 1))]
DiningData_2023 = DDReduced[(date_col >= pd.Timestamp(2023, 1, 1)) & (date_col < pd.Timestamp(2024, 1, 1))]

<ipython-input-18-3a296608f43f>:8: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  DiningData_2019 = DDReduced[(date_col >= pd.Timestamp(2019, 1, 1)) & (date_col < pd.Timestamp(2020, 1, 1))]
<ipython-input-18-3a296608f43f>:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  DiningData_2020 = DDReduced[(date_col >= pd.Timestamp(2020, 1, 1)) & (date_col < pd.Timestamp(2021, 1, 1))]
<ipython-input-18-3a296608f43f>:10: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Us

---
#Analysis

---

In [19]:
total_carbon_emissions_2022 = DiningData_2022['CO2e(kg)'].sum()
total_duke_emissions_2022 = 194346000 #in kg, from 2022 GHG emissions

print("Of what has been calculated, food is ", total_carbon_emissions_2022 / total_duke_emissions_2022 * 100, "% of Duke's total GHG emissions in 2022")

Of what has been calculated, food is  15.819781918627395 % of Duke's total GHG emissions in 2022
